## View Updates

EX: Consider view of all loan data in the loan relation,
hiding the amount attr:

```sql
CREATE VIEW 	branch_loan AS
	SELECT 	branch_name, loan_number
	FROM		loan
```

Adding a tuple to the view like so:
```sql
INSERT INTO 	branch_loan
	VALUES	('L-307', 'La Jolla')
```

leads to the insertion of the tuple `('L-307', 'La Jolla',
null)` into the loan relation.

Any unspecified values will default (if default val exists)
or be null. 

If neither default val exist nor null values
are allowed (via constraints), then the insertion will be
rejected


Updates are only allowed on views with:
- no aggregates 
- no group-by
- no tuple aliases 
- single base table definition
- naturally mapping of updates to underlying base tables

Any other views has no guarantee: might not uniquely transform an update on view --> update on the base table

Hence, most SQL implementations only allow updates on a very restricted class of your definitions


EX: 

```sql
CREATE VIEW	Berto-titles AS
	SELECT	title
	FROM		movie
	WHERE	director='Bertolucci'
```

DELETE a title T in view:
 - delete all tuples with title T from movie (note: there will be tuples for every Actor in the movie)

INSERT a title T in view:
 - insert `<T, "Bertolucci", NULL>` in movie (don't know Actor)

Update "Sky" to "Sheltering Sky" in view
``` sql 
UPDATE movie
	SET		title = 'Sheltering Sky'
	WHERE 	director = 'Bertolucci' 
		AND	title = 'Sky'
```

DBMS performs all these translations under the hood

### More complicated example

```sql
CREATE VIEW 	Same AS
	SELECT	t.theater, s.theater
	FROM		schedule t, schedule s
	WHERE	t.title = s.title
```

Insertion of <Ken, Hillcrest> into Same

> Problem: Cannot be mapped to an update of "Movie" 
(the common title is unknown)

Cause: the view loses information. It uses more information 
in the WHERE than is displayed in the output of SELECT

Similar problems arise for DELETE and UPDATES

Such view updates are **prohibited**

# Assertions

Defines a constraint the database must satisfy

**Syntax**: 
```sql
CREATE ASSERTION 	<assertion-name>
		CHECK	<predicate>
```

When assertion is made, system handles validty checks,
and tests it again on every update that might violate
the assertion

> Note: testing may introduce a significant amount of 
overhead; hence assertions should be used with great care.

Asserting `for all X, P(X)` predicates are achieved in a 
round-about fashion using `not exists X s.t. not P(X)` 

(reminder: assertions are just a Boolean T/F value)

## Using General Assertions

Specify a query that violates the condition, 
then include inside a `NOT EXISTS` clause

Query result must be empty,
not empty --> assertion has been violated


EX: every loan has (at least one) borrower who maintains
an account with a (minimum balance of $1,000.00

```sql
CREATE ASSERTION	balance_constraint CHECK (
	-- 
	NOT EXISTS	(
		-- Finds loans without ANY borrowers that match the condition
		SELECT	*
		FROM	loan
		WHERE 	NOT EXISTS (
			-- Relates loan with borrower, depositor, and account to find
			-- borrowers for each loan that match the condition (min balance)
			SELECT 	*
			FROM 	borrower, depositor, account
			WHERE 	loan.loan_number = borrower.loan_number
				AND	borrower.customer_id = depositor.customer_id
				AND  depositor.account_number = account.account_number
				AND 	account.balance >= 1000.00
			)
		)
	)
```

Innermost query connects all 4 relations first. All relations within this FROM will be used to link together and tied to single tuples at a time, but will be iterated for EVERY tuple from the exterior relation (loan). Thus for every loan, we'll be going through every borrower that exists for that loan.

Then it does a check to filter out any borrower with less than $1,000 (non-compliant borrowers) so that ONLY COMPLIANT borrowers are left over. 

The inner `NOT EXISTS` then determines, for each loan, whether the COMPLIANT BORROWERS list is empty.

The outer `NOT EXISTS` then flips the assertion so that it returns whether or not the COMPLIANT BORROWERS list for each loan is NOT empty.


EX: The sum of all loan amounts for each branch must be less than the sum of all account balances at the branch

```sql
CREATE ASSERTION    sum_constraint CHECK
    (NOT EXISTS (
            SELECT  *
            FROM    branch
            WHERE   (
                SELECT  SUM(amount)
                FROM    loan
                WHERE   loan.branch_name = branch.branch_name
            ) >= (
                SELECT  SUM(amount)
                FROM    account
                where   account.branch_name = branch.branch_name
            )
    ))
```

Another way of reading it, top down :

`NOT EXISTS` will check if contained query is empty.

containd query checks all branches and outputs any where (A) >= (B)

A) aggregate sum of loans within the branch

B) aggregate sum of account balances within the branch

All together it reads: if (the branches where (sum(loans within branch) >= sum(accounts within branch))) is empty

Basically the opposite of branches where the sum of all loan ammounts are greater than or equal to (opp of condition) sum of all account ballances at branch

EX: The salary of an employee must not be greater than the salary of the manager of the departmnet that the employee works for

```sql
CREATE ASSERTION    SALARY_CONSTRAINT
CHECK (NOT EXISTS (             -- Ensure NO violations
    SELECT  *
    FROM    EMPLOYEE E, EMPLOYEE M, DEPARTMENT D
    WHERE   E.SALARY > M.SALARY -- E makes more than M (violates)
        AND E.DNO = D.NUMBER    -- employee (E) is in same department
        AND D.MGRSSN = M.SSN    -- as the manager (M)
))
```

# SQL Triggers

**Objective**: monitor a database and take actions when a condition occurs

**Syntax**: Triggers are expressed in a syntax similar to assertions and include the following:

- event (e.g. an update operation)
- condition
- action (to be taken when the condition is satisfied)